In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/electronics/electronics.csv


In [2]:
path = '/kaggle/input/electronics/electronics.csv'
df = pd.read_csv(path)
df.shape

(1292954, 10)

In [3]:
df.head()

,item_id,user_id,rating,timestamp,model_attr,category,brand,year,user_attr,split
0,0,0,5.0,1999-06-13,Female,Portable Audio & Video,NaN,1999,NaN,0
1,0,1,5.0,1999-06-14,Female,Portable Audio & Video,NaN,1999,NaN,0
2,0,2,3.0,1999-06-17,Female,Portable Audio & Video,NaN,1999,NaN,0
3,0,3,1.0,1999-07-01,Female,Portable Audio & Video,NaN,1999,NaN,0
4,0,4,2.0,1999-07-06,Female,Portable Audio & Video,NaN,1999,NaN,0


In [4]:
from fastai.collab import *

In [5]:
from fastai.tabular.all import *

In [6]:
df = df.sample(10000)
df.shape

(10000, 10)

In [7]:
dls = CollabDataLoaders.from_df(df,user_name='user_id',item_name='category',bs = 64)
dls.show_batch()

,user_id,category,rating
0,841450,Television & Video,3.0
1,199534,Television & Video,4.0
2,1116403,Headphones,1.0
3,521318,Television & Video,5.0
4,597339,Headphones,5.0
5,1122302,Portable Audio & Video,1.0
6,416882,Camera & Photo,5.0
7,886755,Computers & Accessories,5.0
8,1143421,Portable Audio & Video,5.0
9,47780,Accessories & Supplies,4.0


In [8]:
n_users = len(dls.classes['user_id'])
n_category = len(dls.classes['category'])
n_factors = 5 
user_factors = torch.randn(n_users,n_factors)
category_factors = torch.randn(n_category, n_factors)

In [9]:
one_hot_6 = one_hot(6,n_users).float()
user_factors.t()@one_hot_6

tensor([ 0.2114,  0.4590,  0.7140, -2.0231, -0.6213])

In [10]:
user_factors[6]

tensor([ 0.2114,  0.4590,  0.7140, -2.0231, -0.6213])

In [11]:
class example:
    def __init__(self,a):
        self.a = a
    def say(self,x):
        return f'Hello {self.a}{x}.'

In [12]:
ex = example('Subha')
ex.say(' Please come out of that')

'Hello Subha Please come out of that.'

In [13]:
class DotProduct(Module):
    def __init__(self,n_users,n_category,n_factors):
        self.user_factors = Embedding(n_users,n_factors)
        self.category_factors = Embedding(n_category,n_factors)
    def forward (self,x):
        users = self.user_factors(x[:,0])
        categories = self.category_factors(x[:,1])
        return (users*categories).sum(dim=1)

In [14]:
x,y = dls.one_batch()
x.shape,y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [15]:
model = DotProduct(n_users,n_category,50)
learn = Learner(dls, model, loss_func = MSELossFlat())

In [16]:

learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,18.290144,18.380482,00:01
1,16.050697,18.350023,00:00
2,4.544770,18.408499,00:00
3,0.876366,18.400650,00:00
4,0.215113,18.400930,00:00


In [17]:
class DotProduct(Module):
    def __init__(self, n_users, n_category, n_factors,y_range = (0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.category_factors = Embedding (n_category,n_factors)
        self.y_range = y_range
    
    def forward(self,x):
        users = self.user_factors(x[:,0])
        category = self.user_factors(x[:,1])
        return sigmoid_range((users*category).sum(dim=1),*self.y_range)

In [18]:
dls = CollabDataLoaders.from_df(df,user_name='user_id',item_name='category',bs = 64)
dls.show_batch()

,user_id,category,rating
0,366635,Computers & Accessories,5.0
1,247532,Computers & Accessories,4.0
2,86700,Camera & Photo,5.0
3,995939,Home Audio,5.0
4,310304,Computers & Accessories,5.0
5,375817,Headphones,5.0
6,218915,Computers & Accessories,5.0
7,1048619,Home Audio,3.0
8,907806,Portable Audio & Video,4.0
9,217556,Computers & Accessories,4.0


In [19]:
model = DotProduct(n_users,n_category,50)
learn = Learner(dls,model,loss_func = MSELossFlat())
learn.fit_one_cycle(5,5e-5)

epoch,train_loss,valid_loss,time
0,3.586854,3.472761,00:00
1,3.580037,3.472803,00:00
2,3.590483,3.472850,00:00
3,3.598910,3.472879,00:00
4,3.579884,3.472881,00:00


In [20]:
class DotProductBias(Module):
    def __init__(self, n_users, n_category, n_factors, y_range = (0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.category_factors = Embedding(n_category, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.category_bias = Embedding(n_category, 1)
        self.y_range = y_range
        
    def forward(self,x):
        users = self.user_factors(x[:,0])
        category = self.category_factors(x[:,1])
        res = ((users*category).sum(dim=1,keepdim = True))
        res += self.user_bias(x[:,0]) + self.category_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)
        

In [21]:
model = DotProductBias(n_users, n_category, 50)
learn = Learner(dls, model, loss_func = MSELossFlat())
learn.fit_one_cycle(5,5e-5,wd = 0.1)

epoch,train_loss,valid_loss,time
0,3.575401,3.449068,00:00
1,3.561864,3.430459,00:00
2,3.526108,3.416178,00:00
3,3.524380,3.409540,00:00
4,3.543298,3.408453,00:00


In [22]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [23]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,3.259068,2.850517,00:00
1,1.758839,2.219197,00:00
2,0.376145,2.198565,00:00
3,0.076606,2.231431,00:00
4,0.017303,2.232777,00:00


In [24]:
class T(Module):
    def __init__(self):
        self.a =  nn.Parameter(torch.ones(3))
L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [25]:
class T(Module):
    def __init__ (self):
        self.a = nn.Linear(1,3, bias = False)
L(T().parameters())

(#1) [Parameter containing:
tensor([[ 0.5693],
        [ 0.8086],
        [-0.3708]], requires_grad=True)]

In [26]:
t = T()
type(t.a.weight)

torch.nn.parameter.Parameter

In [27]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [28]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [29]:
class DotProductBias(Module):
    def __init__(self, n_users, n_category, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.category_factors = create_params([n_category, n_factors])
        self.category_bias = create_params([n_category])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:,0]]
        category = self.category_factors[x[:,1]]
        res = (users*category).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.category_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)

In [30]:
model = DotProductBias(n_users, n_factors, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
category_bias = learn.model.category_bias.squeeze()
idxs = category_bias.argsort()[:5]
[dls.classes['category'][i] for i in idxs]

In [ ]:
idxs = category_bias.argsort(descending = True)[:5]
[dls.classes['category'][i] for i in idxs]

In [ ]:
g = df.groupby('category')['rating'].count()
top_category = g.sort_values(ascending=False).index.values[:1000]
top_idxs = tensor([learn.dls.classes['category'].o2i[m] for m in top_category])
category_w = learn.model.category_factors[top_idxs].cpu().detach()
category_pca = category_w.pca(3)
fac0,fac1,fac2 = category_pca.t()
idxs = list(range(50))
X = fac0[idxs]
Y = fac2[idxs]
plt.figure(figsize=(12,12))
plt.scatter(X, Y)
for i, x, y in zip(top_category[idxs], X, Y):
    plt.text(x,y,i, color=np.random.rand(3)*0.7, fontsize=11)
plt.show()

In [ ]:
category_factors = learn.model.i_weight.weight
idx = dls.classes['category'].o2i['Silence of the Lambs, The (1991)']
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

In [ ]:
embs = get_emb_sz(dls)
embs

In [ ]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5),n_act = 100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1] + item_sz[1],n_act),
            nn.ReLU(),
            nn.Linear(n_act,1))
        self.y_range = y_range
    
    def forward(self,x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs,dim=1))
        return sigmoid_range(x,*self.y_range)

In [ ]:
model = CollabNN(*embs)

In [ ]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(4, 5e-3, wd=0.01)

In [ ]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)